## Jupyter notebook notes to run: 
### Change the ACCESS_KEY and SECRET_KEY to AWS Credentials. 


In [1]:

%pylab inline
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from lib import XGBHelper as xgbh
import os, pickle
import modin.pandas as pd

from lib.logger import logger
from lib.operari import *
if 'nb_dir' not in globals():
    nb_dir = os.getcwd()
    
%load_ext autoreload
%autoreload 2

import pandas as pd
# import s3fs  #if running on aws
# from pandas import DataFrame # if running with weak gpu
from modin.pandas import DataFrame
import modin.pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# !pip install boto3
from boto3.session import Session

from lib.operari import *

Populating the interactive namespace from numpy and matplotlib


Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.


## IF RUNNING PLOTTING OFFLINE, GOTO END

In [4]:

# # Change this to 
# ACCESS_KEY = 'XXX'
# SECRET_KEY = 'XXX'

# session = Session(aws_access_key_id=ACCESS_KEY,
#                   aws_secret_access_key=SECRET_KEY)
# s3 = session.resource('s3')
# your_bucket = s3.Bucket('bathymetry')

# for s3_file in your_bucket.objects.all():
#     print(s3_file.key)

In [6]:
# Read in data from a subset of a region's dataset.
import random 

def get_data_from_s3(prefix, is_jamstec, is_test):
    
    new_df = df_ = pd.DataFrame()
    
    # 100 corresponds to 20% of a region's dataset
    # 250 corresponds to 50% of a region's dataset
    
    num_samples = 50
    
    if is_jamstec: 
        rand_list = random.sample(range(175), num_samples)
    elif is_test: 
        rand_list = random.sample(range(108), 108)
    else: 
        rand_list = random.sample(range(500), num_samples)

    counter = 0 
    for r in rand_list:   
        if 0 <= r <= 9: 
            new_prefix = prefix[:-1]
        elif 10 <= r <= 99: 
            new_prefix = prefix[:-2]
        elif 100 <= 999: 
            new_prefix = prefix[:-3]
        new_prefix += str(r)
        new_prefix += '.tsv'
        counter += 1
        if counter % 10 == 0: 
            print(new_prefix, " ", counter)
        df = pd.read_csv(new_prefix, sep=' ', header=None)
        new_df = new_df.append(df)
        
    return new_df


In [7]:
# takes fewer lines from each file but looks at more files 
import random 

def get_more_data_from_s3(prefix, is_jamstec, is_test):
    
    new_df = df_ = pd.DataFrame()
    
    # 100 corresponds to 20% of a region's dataset
    # 250 corresponds to 50% of a region's dataset
    
    num_samples = 200
    
    if is_jamstec: 
        rand_list = random.sample(range(175), num_samples)
    elif is_test: 
        rand_list = random.sample(range(108), num_samples)
    else: 
        rand_list = random.sample(range(500), num_samples)

    counter = 0 
    for r in rand_list:   
        if 0 <= r <= 9: 
            new_prefix = prefix[:-1]
        elif 10 <= r <= 99: 
            new_prefix = prefix[:-2]
        elif 100 <= 999: 
            new_prefix = prefix[:-3]
        new_prefix += str(r)
        new_prefix += '.tsv'
        counter += 1
        if counter % 10 == 0: 
            print(new_prefix, " ", counter)
        df = pd.read_csv(new_prefix, sep=' ', header=None, nrows=1000)
        new_df = new_df.append(df)
        
    return new_df

In [ ]:
prefix = 's3://bathymetry/JAMSTEC/JAMSTEC-part00000'
jamstec_df = get_more_data_from_s3(prefix, False, False)
jamstec_df.head()

In [ ]:
prefix = 's3://bathymetry/JAMSTEC2/JAMSTEC2-part00000'
jamstec2_df = get_more_data_from_s3(prefix, True, False)
jamstec2_df.head()

In [ ]:
prefix = 's3://bathymetry/NGDC/NGDC-part00000'
ngdc_df = get_more_data_from_s3(prefix, False, False)
ngdc_df.head()

In [12]:
# prefix = 's3://bathymetry/NGDC/NGDC-part00000'
# prefix.find('NGDC')>0 is not None

In [ ]:
prefix = 's3://bathymetry/SIO/SIO-part00000'
sio_df = get_more_data_from_s3(prefix,  False, False)
sio_df.head()

In [ ]:
prefix = 's3://bathymetry/US_multi/US_multi-part00000'
us_multi_df = get_more_data_from_s3(prefix, False, False)
us_multi_df.head()

In [ ]:
prefix = 's3://bathymetry/test/test-part00000'
test_df = get_more_data_from_s3(prefix, False, True)
test_df.head()

In [ ]:
def processing_df(df):
    cols = ["lon", "lat", "depth", "sigh", "sigd", "SID", "pred", "ID", "(pred-depth)/depth", "d10", 'd20', "d60", "age", "VGG", "rate", "sed", "roughness", "G:T", "NDP2.5m", "NDP5m", "NDP10m", "NDP30m", "STD2.5m", "STD5m", "STD10m", "STD30m", "MED2.5", "MED5m", "MED10m", "MED30m", "D-MED2.5m/STD2.5m", "D-MED5m/STD5m", "D-MED10m/STD10m", "D-MED30m/STD30m", "year", "kind"]
    df.columns=cols
    
    # Count number of good vs. bad cruises, using sigd=9999 for false
    df.loc[(df.sigd != "9999"), 'sigd'] = 1
    df.loc[(df.sigd == "9999"), 'sigd'] = 0
    df = df.drop(columns=['SID', 'ID', 'kind'])

    # Convert all columns to float: 
    new_cols = df.columns
    for c in new_cols: 
        df = df[~df[c].isin(["NaN"])]
        df[c] = pd.to_numeric(df[c])
        
    return df

In [ ]:
jamstec_df = processing_df(jamstec_df)
jamstec2_df = processing_df(jamstec2_df)
ngdc_df = processing_df(ngdc_df)
us_multi_df = processing_df(us_multi_df)
sio_df = processing_df(sio_df)
test_df = processing_df(test_df)

In [ ]:
new_cols = jamstec_df.columns
print(new_cols)

# Create 2D mappings of the dataset using latitude and longitude: 

In [ ]:
colors = ['blue', 'orange', 'green', 'red', 'purple', 'gray']
labels = ["JAMSTEC", "JAMSTEC2", "SIO", "NGDC", "US_MULTI", "TEST"]
list_dfs = [jamstec_df, jamstec2_df, sio_df, ngdc_df, us_multi_df, test_df]

plt.rcParams.update({'font.size': 22})
plt.figure(figsize=(20,10))

for i in range(0, len(list_dfs)):
    print(labels[i])
    lon = list_dfs[i]['lon'].values
    lat = list_dfs[i]['lat'].values
    plt.scatter(lon, lat, c=colors[i], label=labels[i])
                

plt.title('Mappings of different cruise regions')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()

plt.show()


In [ ]:
colors = ['blue', 'orange', 'green', 'red', 'purple', 'gray']
labels = ["JAMSTEC", "JAMSTEC2", "SIO", "NGDC", "US_MULTI", "TEST"]
list_dfs = [jamstec_df, jamstec2_df, sio_df, ngdc_df, us_multi_df, test_df]

fig = plt.figure(figsize=(10,10))
    
%load_ext autoreload
%autoreload 2
    plt.rcParams.update({'font.size': 22})
    

    lon = list_dfs[i]['lon'].values
    lat = list_dfs[i]['lat'].values

    plt.scatter(x=lon, y=lat,c=colors[i], label=labels[i])
    plt.scatter(x=test_df['lon'], y=test_df['lat'],c='gray', label="TEST")


#     plt.title('Mappings of different cruise regions')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.legend()

    plt.show()

# Create 3D mappings of the dataset using latitude, longitude, and year: 

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111, projection='3d')

df = jamstec_df


ax.scatter(df['lon'], df['lat'], df['year'], s=100, c='r', marker='o', label='JAMSTEC')
ax.scatter(test_df['lon'], test_df['lat'], test_df['year'], s=100, c='gray', marker='o', label='TEST')


ax.set_xlabel('Longitude', labelpad=20)
ax.set_ylabel('Latitude', labelpad=20)
ax.set_zlabel('Year', labelpad=20)
ax.dist = 10

plt.legend()
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111, projection='3d')

df = jamstec2_df


ax.scatter(df['lon'], df['lat'], df['year'], s=100, c='r', marker='o', label='JAMSTEC2')
ax.scatter(test_df['lon'], test_df['lat'], test_df['year'], s=100, c='gray', marker='o', label='TEST')


ax.set_xlabel('Longitude', labelpad=20)
ax.set_ylabel('Latitude', labelpad=20)
ax.set_zlabel('Year', labelpad=20)
ax.dist = 10

plt.legend()
plt.show()

# generate plot

In [14]:
# data_floder = input("What is your Bathymetry_Data folder called? ")
# data_folder = 'Bathymetry_Data'
# data_dir_list = find_files(filename=data_folder, search_path = nb_dir + '/..')
# print(data_dir_lst)

# data_fn = 'test/test-part00107.tsv'
data_fn = 'NGDC/NGDC-part00000.tsv'
model_fn = 'NGDC_model.pkl'

def model_dir(s): return nb_dir + f'/../runtime_models/{s}'
def data_dir(fn): return nb_dir + f'/../Bathymetry_Data/{fn}'

file_name = model_dir(model_fn)
xgb_model_loaded = pickle.load(open(file_name, "rb"))

dropped_cols = [3,4,5,35]
df = pd.read_csv(data_dir(data_fn), header=None, sep=' ')
# .rename(
#     columns={0:'lat', 1:'long'}).drop(columns=dropped_cols)

In [15]:
# df = get_df(file_name)
df = processing_df(df)

In [19]:
# data_dir(file_name)
# file_name
df.head()

,lon,lat,depth,sigh,sigd,pred,(pred-depth)/depth,d10,d20,d60,...,STD30m,MED2.5,MED5m,MED10m,MED30m,D-MED2.5m/STD2.5m,D-MED5m/STD5m,D-MED10m/STD10m,D-MED30m/STD30m,year
0,-60.94230,39.17754,-4974,0,1,-4990,0.003217,0.274731,0.319196,0.344853,...,749.505847,0.564557,-15.65930,-329.342,97.5445,0.016712,-0.109036,-0.722837,0.130145,2003
1,-60.96140,39.17466,-4936,0,1,-4990,0.010940,0.251237,0.294879,0.336107,...,757.526279,13.182600,1.51866,-381.600,136.5390,0.235613,0.009291,-0.798527,0.180243,2003
2,-60.95665,39.17576,-5115,0,1,-4990,0.024438,0.259644,0.300137,0.337617,...,755.275773,-156.329000,-167.77900,-532.793,-42.7973,-3.148620,-1.063390,-1.133810,-0.056664,2003
3,-60.94819,39.17408,-4980,0,1,-4990,0.002008,0.320955,0.341921,0.351400,...,754.292305,-18.408500,-28.92580,-379.693,92.4795,-0.440272,-0.178749,-0.789339,0.122604,2003
4,-60.94413,39.17424,-4946,0,1,-4990,0.008896,0.328796,0.350331,0.354470,...,752.934078,18.800400,6.53966,-331.169,126.3830,0.496806,0.041181,-0.691294,0.167855,2003


In [18]:
colors = ['red', 'gray']
labels = ["NGDC","TEST"]
list_dfs = [ngdc_df, test_df]

plt.rcParams.update({'font.size': 26})
plt.figure(figsize=(20,10))

for i in range(0, len(list_dfs)):
    #     print(labels[i])
    lon = list_dfs[i]['lon'].values
    lat = list_dfs[i]['lat'].values
    plt.scatter(lon, lat, c=colors[i], label=labels[i])
                

plt.title('Mappings of different cruise regions')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()

plt.show()


NameError: name 'jamstec_df' is not defined

/home/timothytyree/Documents/GitHub/bathymetry/notebooks


In [20]:
region='test'
lst = []
for f in os.walk(f"{nb_dir}/../Bathymetry_Data/{region}/"):
    lst.append(f)

In [23]:
len(lst[0][0])

81

In [25]:
len(lst[0][1])

0

In [26]:
len(lst[0][2])

108

In [28]:
print(lst[0][2][:10])

['test-part00057.tsv', 'test-part00040.tsv', 'test-part00094.tsv', 'test-part00084.tsv', 'test-part00081.tsv', 'test-part00039.tsv', 'test-part00002.tsv', 'test-part00097.tsv', 'test-part00071.tsv', 'test-part00031.tsv']
